# Get Soup

In [1]:
from bs4 import BeautifulSoup
import requests
import regex as re

page = requests.get("https://www.laughingplace.com/w/p/disneyland-current-wait-times/")

In [2]:
dland = BeautifulSoup(page.content, 'html.parser')

In [3]:
page2 = requests.get("https://www.laughingplace.com/w/p/disney-california-adventure-current-wait-times/")

In [4]:
dca = BeautifulSoup(page2.content, 'html.parser')

## Get Attractions Table

In [124]:
attractions, waits = [], []

In [125]:
check = dland.find('div', class_='header').text
check

'Operating Hours For Fri, Oct 8: 8a-11p\n \nLast Check at 3:25pm on Friday'

In [126]:
check = check.split('\n')

In [127]:
pattern = 'Operating Hours For '
date_string = re.sub(pattern, '', check[0])
date_string = date_string[5:11]
date = re.sub('[^A-Za-z0-9]+', ' ', date_string).strip()
date

'Oct 8'

In [128]:
pattern = 'Last Check at '
mod_string = re.sub(pattern, '', check[2])
mod_string = mod_string[:6]
time = re.sub('[^A-Za-z0-9]+', '', mod_string).strip()
time

'325pm'

In [129]:
hours_string = check[0][31:]
hours_string=hours_string.strip()
hours_string

'8a-11p'

In [130]:
total_date=date+' '+hours_string+' @'+time
total_date

'Oct 8 8a-11p @325pm'

# Disneyland

## Attraction Names

In [131]:
first = dland.find('table', class_='lp_attraction')

In [132]:
attraction_names = first.find_all('a')

In [133]:
for item in attraction_names:
    attractions.append(item.text)

In [134]:
attractions.insert(16, 'Haunted Mansion')

In [135]:
attractions.insert(29, "Oga's Cantina")

## Wait Times

In [136]:
wait_times = first.find_all("span", id=re.compile("f"))

In [137]:
for item in wait_times:
    
    if item.find('div', class_='waiticon'):
        raw_time = item.find('td')
        time = raw_time.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
    else:
        time = item.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
        
    waits.append(time)

# DCA

## Attraction Names

In [138]:
first = dca.find('table', class_='lp_attraction')

In [139]:
attraction_names = first.find_all('a')

In [140]:
for item in attraction_names:
    attractions.append(item.text)

In [141]:
attractions.insert(46, 'Carthay Circle')

In [142]:
attractions.insert(55, 'Lamplight Lounge')

In [143]:
attractions.insert(57, 'Magic Key Terrace')

In [144]:
attractions.append('Web Slingers')

## Wait Times

In [145]:
import regex as re

wait_times = first.find_all("span", id=re.compile("f"))
wait_times

[<span id="f_the_bakery_tour">Closed</span>,
 <span id="f_carthay_circle_restaurant">Open</span>,
 <span id="f_golden_zephyr"><span class="statusdown">Down</span></span>,
 <span id="f_goofys_sky_school">35 minutes</span>,
 <span id="f_grizzly_river_run">25 minutes</span>,
 <span id="f_guardians_of_the_galaxy__mission_breakout">Closed</span>,
 <span id="f_incredicoaster">30 minutes</span>,
 <span id="f_inside_out_emotional_whirlwind">20 minutes</span>,
 <span id="f_jessies_critter_carousel">5 minutes</span>,
 <span id="f_jumpin_jellyfish">10 minutes</span>,
 <span id="f_lamplight_lounge___boardwalk_dining">35 minutes</span>,
 <span id="f_the_little_mermaid__ariels_undersea_adventure">15 minutes</span>,
 <span id="f_magic_key_terrace_walk_up_only">Open</span>,
 <span id="f_mickeys_philharmagic"><span class="statusdown">Down</span></span>,
 <span id="f_monsters_inc_mike__sulley_to_the_rescue">
 <div class="waiticon">
 <table>
 <tr>
 <td>40 minutes</td>
 <td><img sizes="(max-width: 37px) 1

In [146]:
for item in wait_times:
   
    if item.find('div', class_='waiticon'):
        raw_time = item.find('td')
        time = raw_time.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
    else:
        time = item.text[:3].strip(' ')
        if ' ' in time:
            time = time[:1]
        
    waits.append(time)

## DataFrame

In [147]:
import pandas as pd

attraction_waits = pd.DataFrame(waits, index=attractions, columns=[total_date])
attraction_waits

,Oct 8 8a-11p @325pm
Alice in Wonderland,25
Astro Orbitor,35
Autopia,40
Big Thunder Mountain Railroad,Ref
Buzz Lightyear Astro Blasters,25
...,...
Redwood Creek Challenge Trail,Ope
Silly Symphony Swings,20
Soarin’ Around the World,35
Toy Story Midway Mania!,45


In [148]:
attraction_waits.to_pickle('attraction_waits.pkl')